<a href="https://colab.research.google.com/github/Shristi1104/NETWORK_SIMULATOR/blob/main/cisco_problem_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

main.c

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include "parser.h"
#include "topology.h"
#include "validation.h"
#include "simulation.h"

int main(int argc, char *argv[]) {
    printf("=== Network Simulator Tool ===\n");

    if (argc < 2) {
        printf("Usage: %s <config-directory>\n", argv[0]);
        return 1;
    }

    char *configDir = argv[1];


    parse_configs(configDir);


    build_topology();


    validate_topology();


    run_simulation();

    return 0;
}

parser.c

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <dirent.h>
#include "parser.h"

#define MAX_PATH_LEN 512   // safer buffer size for file paths

void parse_configs(const char *configDir) {
    DIR *dir;
    struct dirent *entry;

    dir = opendir(configDir);
    if (!dir) {
        perror("Failed to open config directory");
        exit(EXIT_FAILURE);
    }

    printf("Parsing config files in directory: %s\n", configDir);

    while ((entry = readdir(dir)) != NULL) {
        // skip "." and ".."
        if (strcmp(entry->d_name, ".") == 0 || strcmp(entry->d_name, "..") == 0) {
            continue;
        }

        char filepath[MAX_PATH_LEN];
        int written = snprintf(filepath, sizeof(filepath), "%s/%s", configDir, entry->d_name);

        // Check for truncation
        if (written < 0 || written >= sizeof(filepath)) {
            fprintf(stderr, "Error: file path too long, skipping: %s/%s\n",
                    configDir, entry->d_name);
            continue;
        }

        FILE *file = fopen(filepath, "r");
        if (!file) {
            perror("Failed to open config file");
            continue; // skip instead of crashing
        }

        printf("  Loaded config: %s\n", filepath);

        // Example: just read line by line for now
        char line[256];
        while (fgets(line, sizeof(line), file)) {
            printf("    %s", line); // display contents
        }

        fclose(file);
    }

    closedir(dir);
}

topology.c

In [ ]:
#include "topology.h"
#include <stdio.h>
#include <string.h>

// Define global variables
struct Router routers[100];
int routerCount = 0;

void build_topology() {
    printf("Building network topology...\n");

    // Example setup: you can expand this later
    routerCount = 2;

    routers[0].id = 1;
    strncpy(routers[0].name, "Router-A", sizeof(routers[0].name) - 1);
    routers[0].name[sizeof(routers[0].name) - 1] = '\0';

    routers[1].id = 2;
    strncpy(routers[1].name, "Router-B", sizeof(routers[1].name) - 1);
    routers[1].name[sizeof(routers[1].name) - 1] = '\0';

    printf("Topology built with %d routers\n", routerCount);
}


validation.c

In [ ]:
#include <stdio.h>
#include <string.h>
#include "parser.h"

extern RouterConfig routers[];
extern int routerCount;

void validate_topology() {
    printf("\n--- Validating Topology ---\n");

    // Check duplicate IPs
    for (int i = 0; i < routerCount; i++) {
        for (int j = i + 1; j < routerCount; j++) {
            if (strcmp(routers[i].ip, routers[j].ip) == 0) {
                printf("Error: Duplicate IP %s between %s and %s\n",
                       routers[i].ip, routers[i].routerName, routers[j].routerName);
            }
        }
    }
}


simulation.c

In [ ]:
#include <stdio.h>
#include "parser.h"

extern RouterConfig routers[];
extern int routerCount;

void run_simulation() {
    printf("\n--- Running Day-1 Simulation ---\n");
    for (int i = 0; i < routerCount; i++) {
        printf("%s sends ARP and OSPF Hello on IP %s\n",
               routers[i].routerName, routers[i].ip);
    }
}


OUTPUT

In [ ]:
kiit@Shristi:~/network-simulator/src$ ./simulator ../Conf
=== Network Simulator Tool ===
Parsing config files in directory: ../Conf
Building network topology...
Topology built with 2 routers

--- Validating Topology ---

--- Running Day-1 Simulation ---
 sends ARP and OSPF Hello on IP -A
Router-B sends ARP and OSPF Hello on IP
kiit@Shristi:~/network-simulator/src$